# Capstone 1 Project: Machine Learning

The goal is to predict production of different crops at county level. The data set is combined data otbained from USDA, BLS and NOAA. 

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
import os

fileDir = os.path.dirname(os.path.abspath(''))

Reading required data.

In [2]:
data_file_path = os.path.join(fileDir, 'Data\Crop_Pivoted.xlsx')
data_df = pd.read_excel(data_file_path)

#removing all spaces in the column names, since some encoders don't really work well with spaces.
data_df=data_df.rename(columns={'ACRES PLANTED':'ACRES_PLANTED','ACRES HARVESTED':'ACRES_HARVESTED','Yearly Unemployment Rate':'Yearly_Unemployment_Rate'})

data_df.head()

,CROP,Year,State,County,ACRES_PLANTED,ACRES_HARVESTED,PRODUCTION,YIELD,PRICE,Yearly_Unemployment_Rate,avg_temp,max_temp,min_temp,percipitation
0,BARLEY,1984,ARIZONA,COCHISE,2100,2100,8467200,4032.000000,0.062500,5.175000,44.1,57.2,30.8,1.53
1,BARLEY,1985,ARIZONA,COCHISE,900,900,4200000,4666.666667,0.058750,6.333333,42.2,53.2,31.3,1.95
2,BARLEY,1986,ARIZONA,COCHISE,500,500,2016000,4032.000000,0.051458,6.925000,48.1,65.0,31.2,0.15
3,BARLEY,1987,ARIZONA,COCHISE,500,500,2304000,4608.000000,0.045625,6.450000,43.0,58.4,27.5,0.54
4,BARLEY,1989,ARIZONA,COCHISE,500,400,2116800,5292.000000,0.064583,5.358333,42.8,56.8,28.9,1.32


Making few adjustment to data, dropping not required columns, adding previous year values and combining columns.

In [3]:
# Since Yield is a calculated filed from production (yield=production/acres harvested). 
# Predicting production is the key here. Hence we will ignore yield for now.
# And Acers harvested gives a direct hint of production. Dropping 
crop_df = data_df.drop(columns=['YIELD','ACRES_HARVESTED'])

# Since a county name can be repeated between states for a given year. 
# Combing Sate and County columns.
crop_df.loc[:,'State-County']=crop_df.loc[:,'State']+' '+crop_df.loc[:,'County']

#sorting before building previous year production column
crop_df = crop_df.sort_values(by=['CROP','State','County','Year'])

# Capturing previous year production
crop_df.loc[:,'PREV_YEAR_PROD']=crop_df.groupby(['CROP','State-County'])['PRODUCTION'].shift(1)
crop_df.loc[:,'PREV_YEAR_PRICE']=crop_df.groupby(['CROP','State-County'])['PRICE'].shift(1)

# Calculating means of previous production
crop_df.loc[:,'MEAN'] = pd.DataFrame(crop_df.groupby(['CROP','State-County'])['PREV_YEAR_PROD'].expanding().mean()).reset_index(level=[0,1])['PREV_YEAR_PROD']

#dropping rows where previous production values are null. Those are first rows for county for a crop.
crop_df.dropna(axis='rows',inplace=True)

crop_df.reset_index(inplace=True)

#dropping duplicate information
crop_df.drop(columns=['State','County','index','PRICE'],inplace=True)
crop_df.head()

,CROP,Year,ACRES_PLANTED,PRODUCTION,Yearly_Unemployment_Rate,avg_temp,max_temp,min_temp,percipitation,State-County,PREV_YEAR_PROD,PREV_YEAR_PRICE,MEAN
0,BARLEY,1985,900,4200000,6.333333,42.2,53.2,31.3,1.95,ARIZONA COCHISE,8467200.0,0.062500,8467200.0
1,BARLEY,1986,500,2016000,6.925000,48.1,65.0,31.2,0.15,ARIZONA COCHISE,4200000.0,0.058750,6333600.0
2,BARLEY,1987,500,2304000,6.450000,43.0,58.4,27.5,0.54,ARIZONA COCHISE,2016000.0,0.051458,4894400.0
3,BARLEY,1989,500,2116800,5.358333,42.8,56.8,28.9,1.32,ARIZONA COCHISE,2304000.0,0.045625,4246800.0
4,BARLEY,1991,1400,6864000,5.966667,43.9,56.3,31.4,1.55,ARIZONA COCHISE,2116800.0,0.064583,3820800.0


There are two categorical columns that needs to be encoded, since regression doesn't work well with text values.

### Prediction with OneHot Encoder

This encoder cannot be used since increases the dimensionality of the data significantly.

In [4]:
from sklearn.preprocessing import OneHotEncoder
# encoding categorical values of CROP and State-County
oneH_encoded_df = crop_df.copy()
oneH_encoded_df=pd.get_dummies(oneH_encoded_df,prefix=None)
print(oneH_encoded_df.columns)
del oneH_encoded_df

Index(['Year', 'ACRES_PLANTED', 'PRODUCTION', 'Yearly_Unemployment_Rate',
       'avg_temp', 'max_temp', 'min_temp', 'percipitation', 'PREV_YEAR_PROD',
       'PREV_YEAR_PRICE',
       ...
       'State-County_WYOMING NATRONA', 'State-County_WYOMING NIOBRARA',
       'State-County_WYOMING PARK', 'State-County_WYOMING PLATTE',
       'State-County_WYOMING SHERIDAN', 'State-County_WYOMING SWEETWATER',
       'State-County_WYOMING TETON', 'State-County_WYOMING UINTA',
       'State-County_WYOMING WASHAKIE', 'State-County_WYOMING WESTON'],
      dtype='object', length=2742)


### Hashing categorical values

Using hashing seems to be a good option, since it keeps the hashed columns independent of any other variables in the data with small number of extra columns. But some information about the original columns might have lost.<br>
We create three groups of features, all features, features with no hints about the production value (so excluding Acers planted, Previous year production and mean historical production) and then features with limited providing hint ( adding back Acers planted).

In [5]:
from category_encoders.hashing import HashingEncoder

h_encoded_df = crop_df.copy()
hasher = HashingEncoder(return_df=True)
h_encoded_df = hasher.fit_transform(crop_df)
print(h_encoded_df.head())

#Creating feature list
hash_features = list(h_encoded_df.columns)
hash_features.remove('PRODUCTION')

#Features that has no hint of previous year production
hintless_hashFeatures = hash_features.copy()
hintless_hashFeatures.remove('PREV_YEAR_PROD')
hintless_hashFeatures.remove('MEAN')
small_hint_hashFeatures=hintless_hashFeatures.copy()
hintless_hashFeatures.remove('ACRES_PLANTED')

hash_features_set = [hash_features,small_hint_hashFeatures,hintless_hashFeatures]

   col_0  col_1  col_2  col_3  col_4  col_5  col_6  col_7  Year  \
0      0      0      0      1      0      0      1      0  1985   
1      0      0      0      1      0      0      1      0  1986   
2      0      0      0      1      0      0      1      0  1987   
3      0      0      0      1      0      0      1      0  1989   
4      0      0      0      1      0      0      1      0  1991   

   ACRES_PLANTED  PRODUCTION  Yearly_Unemployment_Rate  avg_temp  max_temp  \
0            900     4200000                  6.333333      42.2      53.2   
1            500     2016000                  6.925000      48.1      65.0   
2            500     2304000                  6.450000      43.0      58.4   
3            500     2116800                  5.358333      42.8      56.8   
4           1400     6864000                  5.966667      43.9      56.3   

   min_temp  percipitation  PREV_YEAR_PROD  PREV_YEAR_PRICE       MEAN  
0      31.3           1.95       8467200.0         0.06

In [23]:
from sklearn.model_selection import GridSearchCV

def test_model(features,data, model_type):
    print('Features:\n',features)
    if model_type=='random_forest':
        #testing number of features each tree gets, and number of trees.
        parameters = {'max_features':['sqrt','auto','log2'],'n_estimators':[200,400,600]}
        #Setting random state for consistance results.
        model = RandomForestRegressor(random_state=10, n_jobs=-1)
    elif model_type=='linear':
        #testing effect of normalization
        parameters = {'normalize':['True','False']}
        model = LinearRegression(n_jobs=-1,fit_intercept=False)

    # # Spliting train and test data sets.
    X=data[features]
    y=data['PRODUCTION']
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

    clf = GridSearchCV(model, parameters, cv=3)
    clf.fit(X_train[features],y_train)
    
    print('\tBest parameters: ',clf.best_params_)
    print('\tBest scores: ',clf.best_score_)
    if model_type=='random_forest':
        print('\tFeature importance:',np.around(clf.best_estimator_.feature_importances_,4)*100)
    elif model_type=='linear':
        print('\tCoefficients: ',clf.best_estimator_.coef_)
    print('\tTest Score:',clf.score(X_test[features],y_test))
    print('='*50)
    return clf

With GridSearchCV with a random forest model, the results are clear, having no hint at all the prediction suffers. While using all the features the model predicts with a high score. Removing previous year's production and historical means, the model adjusts and put heavy emphasis on Acres Planted. 

In [7]:
for feature in hash_features_set:
    clf = test_model(feature, h_encoded_df,'random_forest')

Features:
 ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'Year', 'ACRES_PLANTED', 'Yearly_Unemployment_Rate', 'avg_temp', 'max_temp', 'min_temp', 'percipitation', 'PREV_YEAR_PROD', 'PREV_YEAR_PRICE', 'MEAN']
	Best parameters:  {'max_features': 'sqrt', 'n_estimators': 600}
	Best scores:  0.9788622385855833
	Feature importance: [3.000e-02 3.000e-02 5.000e-02 5.000e-02 5.900e-01 3.000e-02 2.600e-01
 4.000e-02 1.570e+00 1.886e+01 5.900e-01 1.860e+00 1.280e+00 1.200e+00
 9.000e-01 3.784e+01 2.070e+00 3.273e+01]
	Test Score: 0.9814279060580565
Features:
 ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'Year', 'ACRES_PLANTED', 'Yearly_Unemployment_Rate', 'avg_temp', 'max_temp', 'min_temp', 'percipitation', 'PREV_YEAR_PRICE']
	Best parameters:  {'max_features': 'auto', 'n_estimators': 400}
	Best scores:  0.9676966701574446
	Feature importance: [5.000e-02 5.000e-02 5.000e-02 4.000e-02 8.300e-01 3.000e-02 1.200e-01
 4.000e-02 6.330e+00 7.049e+0

## Linear Regression

Using linear regression instead produces a less favorable model compared to a random forest. But overall the importance of the features remained the same, with a small change in hashed out columns.

In [24]:
for features in hash_features_set:
   clf = test_model(features,h_encoded_df,'linear')

Features:
 ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'Year', 'ACRES_PLANTED', 'Yearly_Unemployment_Rate', 'avg_temp', 'max_temp', 'min_temp', 'percipitation', 'PREV_YEAR_PROD', 'PREV_YEAR_PRICE', 'MEAN']
	Best parameters:  {'normalize': 'True'}
	Best scores:  0.9490688895570368
	Coefficients:  [-1.31350792e+09 -1.31295247e+09 -1.31000101e+09 -1.31313624e+09
 -1.31352775e+09 -1.31214266e+09 -1.31511993e+09 -1.31279910e+09
  1.30723315e+06  4.86445100e+02 -5.46321811e+05 -2.06490918e+06
  2.00692506e+06 -1.41205881e+05  3.70549420e+05  5.21866197e-01
 -5.86383444e+07  5.03775164e-01]
	Test Score: 0.9466408897269597
Features:
 ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'Year', 'ACRES_PLANTED', 'Yearly_Unemployment_Rate', 'avg_temp', 'max_temp', 'min_temp', 'percipitation', 'PREV_YEAR_PRICE']
	Best parameters:  {'normalize': 'True'}
	Best scores:  0.7190289070329374
	Coefficients:  [-4.83392737e+09 -4.82104651e+09 -4.77740223e+09

In [22]:
from statsmodels.formula.api import ols
m = ols('PRODUCTION ~ Year+ACRES_PLANTED+PREV_YEAR_PROD+MEAN+PREV_YEAR_PRICE+Yearly_Unemployment_Rate+avg_temp+max_temp+min_temp+percipitation',h_encoded_df).fit()
print(m.summary())
del m

                            OLS Regression Results                            
Dep. Variable:             PRODUCTION   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.949
Method:                 Least Squares   F-statistic:                 2.539e+05
Date:                Fri, 03 May 2019   Prob (F-statistic):               0.00
Time:                        12:25:15   Log-Likelihood:            -2.6924e+06
No. Observations:              137533   AIC:                         5.385e+06
Df Residuals:                  137522   BIC:                         5.385e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               